**Name** - Manu Mathew
**StudentID** - 8990691

**Objective** - The primary objective of the Lab is to build and compare image classification model that can accurately distinguish the images of cats and dogs. In this notebook, 
- We first prepare a manageable dataset of 5000 images where 2500 images are of dogs and 2500 images are of cats.
- Then we perform the exploratory data analysis (EDA) to understand the dataset characteristics and identify data quality issues.
- Train and evaluate two deep learning models:
    - A custom-defined Convolutional Neural Network (CNN).
    - A fine-tuned VGG16 model pre-trained on ImageNet.
- Explore the relative performance of the models.

`Approach`: 
- Data:
    Use 5000 images from Kaggle and organize them into train/test folders.

- Preprocessing:
    Resize images, normalize pixel values, apply data augmentation, and split into train/validation/test sets.

- EDA:
    Visualize sample images, check class distribution, and analyze pixel intensity.

- Model 1 - Custom CNN:
    Build and train a basic CNN with conv, pooling, and dense layers using binary classification.

- Model 2 - Fine-Tune VGG16:
    Load pre-trained VGG16 (without top), freeze base, add custom top layers, and train with the same data.

- Evaluation:
    Compare both models using accuracy, confusion matrix, precision, recall, F1-score, and PR curve.

- Error Analysis:
    Show and review misclassified images to find patterns in errors.

- Conclusion:
    Summarize which model performed better and suggest possible improvements.